# Preambolo
Ciò che cerco di fare con il seguente esperimento è di indurre 5 alberi di decisione, uno per ogni classe: ciascun albero sarà allenato sul training set così da classificare un punto come appartenente alla classe associata all'albero stesso o meno (classificazione one vs all). In particolare prevedo l'eventuale utilizzo della classe GridSearchCV così da indurre gli alberi caratterizzati da hyper-parametri ottimali rispetto ad una data score (come ad esempio l'accuratezza). Una volta che gli alberi one vs all con hyper-parametri ottimali sono stati indotti e le loro performance valutate  attraverso cross validation sul training set, procedo a ricavare per ciascun albero le probabilità che i punti del test set appartengano o meno alla classe associata all'albero. Per ciascun punto del test set determinerò qual è la classe a cui corrisponde la massima probabilità tra quelle appena ricavate, classificherò il punto come appartenente a quella classe e valuterò l'accuratezza di questo metodo di classificazione.

## Moduli importati

In [33]:
import pandas as pd
import numpy as np
from sklearn import tree
from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_score, StratifiedKFold
from sklearn.metrics import classification_report, accuracy_score

## Importo il dataset

In [34]:
df = pd.read_excel('Db_parotide_Def_REV_Samuel.xlsx')
df['ADC'] = df['ADC'].apply(lambda s: str(s).replace(',', '.')).astype(float)
df.iat[0, 6] = 3
df.iat[1, 6] = 3
df.iat[20, 6] = 3
df.head()

,ID,eta,Segni macro malignità,ADC,TIC type,T2,Output Algoritmo,COD ISTO,ISTO DEFINITIVO-PULITO,ISTO DEFINITIVO,...,CONDORD CODICE FNAC-ISTO DIAGN,CONCORD multiRMN-ISTO B/M,CONCORD multiRMN-ISTO DIAGN,CONCORD FNAC-RMN,FNAC,DATA FNAC,ESITO FNAC,COD FNAC,MILAN,DATA INTERVENTO
0,4,33,0,2.60,D,0,3,1,CARCINOMA,CARCINOMA SQUAMOCELLULARE G2,...,1,1,1,1,0,27/12/2017,Positivo per CTM. Carcinoma squamocellulare o ...,3,6,24/01/2018
1,10,50,0,2.44,D,0,3,1,CARCINOMA,CARCINOMA ADENOIDECISTICO G3,...,1,1,1,1,0,27/12/2016,carcinoma adenoidecistico,3,6,27/11/2019
2,9,81,1,0.70,C,1,1,1,CARCINOMA,CARCINOMA SQUAMOCELLULARE G2,...,1,1,1,1,0,21/03/2019,"Positivo per CTM, carcinoma squamocellulare",3,6,15/05/2019
3,1,28,0,1.60,A,1,3,1,CARCINOMA,CARCINOMA DEI DOTTI SALIVARI,...,1,1,1,1,0,2016-06-10 00:00:00,sospetto per CTM,5,5,25/01/2017
4,7,51,0,1.40,A,0,3,1,METASTASI DA CARCINOMA SQUAMOCELLULARE ALLA PA...,METASTASI DA CARCINOMA SQUAMOCELLULARE ALLA PA...,...,1,1,1,1,0,13/09/2018,Positivo per CTM. Carcinoma squamocellulare,3,6,2018-10-10 00:00:00


## Samples e labels

In [35]:
X_not_encoded = df.loc[:,'eta':'T2'].copy()
X = pd.get_dummies(X_not_encoded, columns = ['TIC type'])
y = df['COD ISTO'].copy()

## Esperimento

In [36]:
# X --> samples

# y --> labels

# tuned_parameters --> dizionario dove specificare gli hyper-paramethers da ottimizzare e il range di valori da 
# considerare per ciascuno di essi 

# test_size --> la frazione del dataset che costituisce il test set (stratificato). di default è 0.2

# model_selection --> di default usa GridSearchCV. Se settato a None usa l'albero 
#di default costruito con DecisionTreeClassifier(),


# score --> la score rispetto alla quale voglio ottimizzare gli hyper-paramethers. Di default è l'accuratezza

# cv --> il numero di fold della cross validation utilizzata per valutare le performance degli alberi one vs all

# random_state_tree --> il seed dei DT, per riproducibilità. Di default è None

# n_it --> il numero di iterazioni

# verbose --> di default è False e determina un report relativamente compatto.
#Se posto True mostra un report contenente informazioni anche sulle performance di
# ciascun albero one vs all allenato nel corso delle iterazioni

def esperimento(X, y, tuned_parameters, test_size = 0.2, model_selection = 'gridsearch',
                score = 'accuracy', cv= 2, random_state_tree = None, n_it = 10, verbose = False):
    
    proba_scores = []
    
    for n in range(n_it):
        print('### ITERATION %d ###\n\n' %(n+1))
        
        #metto shuffle a True senza specificare il random_state per ottenere split random ad ogni iterazione
        #pongo stratify = y per ottenere un test set stratificato dove è presente almeno un sample delle
        #classi minoritarie
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = test_size, 
                                                            shuffle = True, stratify = y)
     
        #dizionari che contengono le labels binarie da utilizzare per allenare i DT one vs all
        bin_y_train = dict()
        bin_y_test = dict()
        for i in range(1,6):
            bin_y_train[i] = [1 if x == i else 0 for x in y_train ]
            bin_y_test[i] = [1 if x == i else 0 for x in y_test ]
        
        #dizionario per contenere le probabilità ottenute da ciascun DT one vs all nell'iterazione corrente
        predict_proba = dict()
        
        #induzione dei 5 DT one vs all
        for n_tree in range(1,6):
            print('# Tuning hyper-parameters DT n.%d for %s #\n' %(n_tree, score))
            
            albero = tree.DecisionTreeClassifier(random_state = random_state_tree)
            
            if model_selection == 'gridsearch':
                clf = GridSearchCV(albero, tuned_parameters, scoring = score, cv = cv, refit = True)
                clf.fit(X_train, bin_y_train[n_tree])
                params = clf.best_params_
                score_ = clf.best_score_
            else:
                
                scores = cross_val_score(albero, X_train, bin_y_train[n_tree], scoring = score, cv = cv )
                clf = albero.fit(X_train, bin_y_train[n_tree])
                score_ = scores.mean()
                params = clf.get_params()
                
            print("Parameters set found:\n")
            print(params)
            print()
            print("Mean cross-validated %s of the estimator: %0.3f\n" %(score, score_))
            if verbose:
                
                print("Detailed classification report:\n")
                print("The model is trained on the training set.\n")
                print("The following scores are computed on the test set.\n")
                y_pred = clf.predict(X_test)
                print(classification_report(bin_y_test[n_tree], y_pred, zero_division = 0))
                
            predict_proba[n_tree] = clf.predict_proba(X_test)[:,1]
            print()
            print('-'*100)

        print('# Aggregazione probabilità #\n')
        array = np.array( [predict_proba[1],  predict_proba[2],  predict_proba[3],  
                           predict_proba[4], predict_proba[5]])
        array = np.transpose(array)
        y_pred_proba = np.argmax(array, axis = 1)+1
        if verbose:
            print(classification_report( y_test, y_pred_proba, zero_division = 0))
        proba_scores.append(accuracy_score(y_test, y_pred_proba))
        
    print('\n\n### Mean final %s: %0.3f ###' %(score, np.mean(proba_scores)))

In [17]:
#Utilizzo gridsearch

seed = 42
tuned_parameters = {'criterion': ['gini', 'entropy'],
                    'max_depth': list(range(2,11)),     
                    'min_samples_leaf': list(range(2,11)),
                    'min_samples_split': list(range(2,11))}

esperimento(X, y, tuned_parameters, random_state_tree = seed)

### ITERATION 1 ###


# Tuning hyper-parameters DT n.1 for accuracy #

Parameters set found:

{'criterion': 'gini', 'max_depth': 2, 'min_samples_leaf': 2, 'min_samples_split': 9}

Mean cross-validated accuracy of the estimator: 0.847


----------------------------------------------------------------------------------------------------
# Tuning hyper-parameters DT n.2 for accuracy #

Parameters set found:

{'criterion': 'gini', 'max_depth': 2, 'min_samples_leaf': 2, 'min_samples_split': 2}

Mean cross-validated accuracy of the estimator: 0.953


----------------------------------------------------------------------------------------------------
# Tuning hyper-parameters DT n.3 for accuracy #

Parameters set found:

{'criterion': 'gini', 'max_depth': 6, 'min_samples_leaf': 3, 'min_samples_split': 8}

Mean cross-validated accuracy of the estimator: 0.646


----------------------------------------------------------------------------------------------------
# Tuning hyper-parameters DT n.4 

Parameters set found:

{'criterion': 'gini', 'max_depth': 2, 'min_samples_leaf': 10, 'min_samples_split': 2}

Mean cross-validated accuracy of the estimator: 0.847


----------------------------------------------------------------------------------------------------
# Tuning hyper-parameters DT n.2 for accuracy #

Parameters set found:

{'criterion': 'gini', 'max_depth': 2, 'min_samples_leaf': 2, 'min_samples_split': 2}

Mean cross-validated accuracy of the estimator: 0.953


----------------------------------------------------------------------------------------------------
# Tuning hyper-parameters DT n.3 for accuracy #

Parameters set found:

{'criterion': 'gini', 'max_depth': 5, 'min_samples_leaf': 2, 'min_samples_split': 2}

Mean cross-validated accuracy of the estimator: 0.529


----------------------------------------------------------------------------------------------------
# Tuning hyper-parameters DT n.4 for accuracy #

Parameters set found:

{'criterion': 'gini', 'max_dept

In [37]:
#Non utilizzo gridsearch, ma semplicemente l'albero di default

esperimento(X, y, tuned_parameters, random_state_tree = seed, model_selection = None)

### ITERATION 1 ###


# Tuning hyper-parameters DT n.1 for accuracy #

Parameters set found:

{'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'random_state': 42, 'splitter': 'best'}

Mean cross-validated accuracy of the estimator: 0.741


----------------------------------------------------------------------------------------------------
# Tuning hyper-parameters DT n.2 for accuracy #

Parameters set found:

{'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'random_state': 42, 'splitter': 'best'}

Mean cross-validated accuracy of the estimator: 0.906


---------

Parameters set found:

{'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'random_state': 42, 'splitter': 'best'}

Mean cross-validated accuracy of the estimator: 0.647


----------------------------------------------------------------------------------------------------
# Tuning hyper-parameters DT n.5 for accuracy #

Parameters set found:

{'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'random_state': 42, 'splitter': 'best'}

Mean cross-validated accuracy of the estimator: 0.918


--------------------------------------------------------------------------------

Parameters set found:

{'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'random_state': 42, 'splitter': 'best'}

Mean cross-validated accuracy of the estimator: 0.906


----------------------------------------------------------------------------------------------------
# Aggregazione probabilità #

### ITERATION 8 ###


# Tuning hyper-parameters DT n.1 for accuracy #

Parameters set found:

{'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'random_state': 42, 'splitter': 'best'}

Mean cross-validated accuracy of the estimator: 0.659


----------------------------

In [31]:
#Altro esperimento con grid search, ma stavolta facendo cv stratificata oltre che fare test split stratificato

strat_k_fold = StratifiedKFold(n_splits=2, shuffle=True, random_state= None)

esperimento(X, y, tuned_parameters, random_state_tree = seed, cv = strat_k_fold )

### ITERATION 1 ###


# Tuning hyper-parameters DT n.1 for accuracy #

Parameters set found:

{'criterion': 'gini', 'max_depth': 2, 'min_samples_leaf': 2, 'min_samples_split': 2}

Mean cross-validated accuracy of the estimator: 0.847


----------------------------------------------------------------------------------------------------
# Tuning hyper-parameters DT n.2 for accuracy #

Parameters set found:

{'criterion': 'gini', 'max_depth': 2, 'min_samples_leaf': 2, 'min_samples_split': 2}

Mean cross-validated accuracy of the estimator: 0.953


----------------------------------------------------------------------------------------------------
# Tuning hyper-parameters DT n.3 for accuracy #

Parameters set found:

{'criterion': 'gini', 'max_depth': 3, 'min_samples_leaf': 6, 'min_samples_split': 2}

Mean cross-validated accuracy of the estimator: 0.624


----------------------------------------------------------------------------------------------------
# Tuning hyper-parameters DT n.4 

Parameters set found:

{'criterion': 'gini', 'max_depth': 2, 'min_samples_leaf': 4, 'min_samples_split': 2}

Mean cross-validated accuracy of the estimator: 0.847


----------------------------------------------------------------------------------------------------
# Tuning hyper-parameters DT n.2 for accuracy #

Parameters set found:

{'criterion': 'gini', 'max_depth': 2, 'min_samples_leaf': 2, 'min_samples_split': 2}

Mean cross-validated accuracy of the estimator: 0.953


----------------------------------------------------------------------------------------------------
# Tuning hyper-parameters DT n.3 for accuracy #

Parameters set found:

{'criterion': 'gini', 'max_depth': 5, 'min_samples_leaf': 2, 'min_samples_split': 2}

Mean cross-validated accuracy of the estimator: 0.589


----------------------------------------------------------------------------------------------------
# Tuning hyper-parameters DT n.4 for accuracy #

Parameters set found:

{'criterion': 'gini', 'max_depth